In [ ]:
# =========================
# SVR – Tuning-Pipeline (analog ET/EN/LGBM)
# =========================
# - USE_DYNAMIC_FI_PIPELINE = False -> Full FE (ifo + optionale Target-Blöcke)
# - USE_DYNAMIC_FI_PIPELINE = True  -> Dynamic FI (rolling Feature Importance Top-N)
# - Modell: src.models.SVR.ForecastModel
# =========================

import os, sys
from pathlib import Path
from itertools import product
import numpy as np
import pandas as pd

# --- 1) Pfad-Setup (wie bei ET/EN/LGBM) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Repo-Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,   # expanding
    EWMA_CORR_SPEC,      # ewm
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.SVR import ForecastModel  # <- dieser Wrapper (s.o.)

# --- 3) Master-Schalter & Meta ---
USE_DYNAMIC_FI_PIPELINE = False
SEED = 42

MODEL_NAME = "svr_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "svr"
outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")

# --- 4) Daten laden ---
y = load_target()             # ΔIP (DatetimeIndex)
X_ifo = load_ifo_features()   # ifo features (für Full FE)

FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print("Dynamic-FI Daten gefunden. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER beim Laden der Dynamic-FI Artefakte:", e)
        print("Bitte erst die FI-Jobs laufen lassen (feature_importance.ipynb).")
        raise
else:
    idx_common = y.index.intersection(X_ifo.index)
    y    = y.loc[idx_common]
    X_ifo = X_ifo.loc[idx_common]
    X_full_lagged = rolling_imp = y_fi = None
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)

# --- 5) Base-Config (Thesis-Policy, wie bei ET) ---
def base_cfg_thesis() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg_thesis()

# --- 6) Corr-Helper (wie ET/EN/LGBM) ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    elif kind == "ewm":
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 7) Grids -------------------------------------------------
if USE_DYNAMIC_FI_PIPELINE:
    # --------- Dynamic FI (nur Modell-HPs + n_features_to_use) ----------
    print("Erstelle HP-Grid für 'Dynamic FI'...")

    N_FEATURES_TO_USE = 20

    # SVR-HPs (konservativ, nicht zu groß):
    kernel_list  = ["rbf", "linear"]
    C_list       = [0.5, 1.0, 2.0]
    epsilon_list = [0.05, 0.1]
    gamma_list   = ["scale", "auto"]   # bei 'linear' wird gamma ignoriert
    tol_list     = [1e-3]
    shrinking_list = [True]
    max_iter_list  = [-1]

    weight_opts = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.98}]

    def build_model_grid_dynamic_fi():
        hp_names = ["kernel","C","epsilon","gamma","tol","shrinking","max_iter"]
        hp_lists = [ kernel_list, C_list, epsilon_list, gamma_list, tol_list, shrinking_list, max_iter_list ]
        grid = []
        for vals in product(*hp_lists):
            base = dict(zip(hp_names, vals))
            base.update({
                "n_features_to_use": N_FEATURES_TO_USE,
                "seed": SEED,
            })
            for w in weight_opts:
                hp = dict(base); hp.update(w); grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    # ----------------- Full FE (analog ET/EN/LGBM) ---------------------
    print("Erstelle HP-Grid für 'Full FE'...")

    # A) FE/DR-Listen
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
        ("ewm",       make_corr_spec("ewm")),
    ]
    lag_candidates_list   = [(1, 2, 3, 6, 12)]
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100, 300]
    redundancy_param_list = [0.90]
    dr_options_list       = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_var_target": 0.95, "pca_kmax": 50},
        {"dr_method": "pls", "pls_components": 8},
    ]

    # B) SVR-HPs (überschaubar)
    kernel_list  = ["rbf", "linear"]
    C_list       = [0.5, 1.0, 2.0]
    epsilon_list = [0.05, 0.1]
    gamma_list   = ["scale", "auto"]
    tol_list     = [1e-3]
    shrinking_list = [True]
    max_iter_list  = [-1]

    # C) Target Blocks & Weighting
    target_block_options = [None, ["AR1"], ["Chronos"], ["TSFresh"]]
    weight_opts          = [{"sample_weight_decay": None}]

    def build_model_grid_full_fe():
        hp_grid = []
        fe_lists = [
            lag_candidates_list,
            top_k_lags_list,
            use_rm3_list,
            k1_topk_list,
            redundancy_param_list,
            dr_options_list,
        ]

        svr_names = ["kernel","C","epsilon","gamma","tol","shrinking","max_iter"]
        svr_lists = [ kernel_list, C_list, epsilon_list, gamma_list, tol_list, shrinking_list, max_iter_list ]

        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                if k1 == 100 and dr_opt["dr_method"] != "none":
                    continue
                base_fe = {
                    "lag_candidates": lags,
                    "top_k_lags_per_feature": k_lags,
                    "use_rm3": rm3,
                    "k1_topk": k1,
                    "redundancy_param": red,
                    **dr_opt,
                    "corr_tag": corr_tag,
                    "corr_spec": corr_spec,
                }
                for v in product(*svr_lists):
                    svr_hp = dict(zip(svr_names, v))
                    for block_set in target_block_options:
                        for w in weight_opts:
                            hp = {
                                **base_fe,
                                **svr_hp,
                                "seed": SEED,
                                "target_block_set": block_set,
                                **w,
                            }
                            hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("Optimierte HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")

# --- 8) Stage A/B Lauf (wie ET/EN/LGBM) ------------------------------------------
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,   # Platzhalter
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(10, len(model_grid)),
            min_survivors_per_block=2,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(10, len(model_grid)),
            min_survivors_per_block=2,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")
